<a href="https://colab.research.google.com/github/Lenaami/SpeechSynthesisLabs/blob/main/SS_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
path = '/content/gdrive/My Drive/Colab Notebooks/Синтез речи/'

In [3]:
book1 = 'r_hod.Result.xml'
book2 = 'tropa.Result.xml'
book3 = 'whtguard.Result.xml'

# Парсинг xml

In [4]:
import xml.etree.ElementTree as ET 
import numpy as np

In [5]:
def isNone(a):
    return int(a) if a is not None else -1

In [6]:
# n-граммы
def add_n_gramm(n_gramm, X):
    zero_word = (-1) * np.ones(X.shape[1], dtype = int)

    X_gram = X.copy()

    for i in range(n_gramm):
        X_gram = np.vstack((zero_word, X_gram, zero_word))

    for i in range(1, n_gramm + 1):
        X = np.hstack((X_gram[(n_gramm - i):-(n_gramm + i), :], X, X_gram[(i + n_gramm):(X_gram.shape[0] - n_gramm + i), :]))
    
    return X

In [7]:
def get_data(file, n_gramm = 0, bySentence = False):
    X = []
    y = []
    words = []

    tree = ET.parse(file)
    root = tree.getroot()

    for snt in root.findall('sentence'):
        features = []
        words_snt = []   
        for wrd in snt.findall('word'): # теги слова       
            words_snt.append(wrd.get('original'))

            feat_wrd = []
            dct = wrd.find('dictitem')
            feat_wrd.append(isNone(dct.get('form'))) 
            feat_wrd.append(isNone(dct.get('subpart_of_speech')))     
            feat_wrd.append(isNone(dct.get('semantics1')))
            feat_wrd.append(isNone(dct.get('semantics2')))        
            feat_wrd.append(isNone(wrd.find('letter').get('flag')))

            features.append(feat_wrd)
        for i, cnt in enumerate(snt.findall('content')): # пунктуация и эмфазы
            features[i].append(isNone(cnt.get('PunktBeg')))
            features[i].append(isNone(cnt.get('PunktEnd')))
            features[i].append(isNone(cnt.get('EmphBeg')))
            features[i].append(isNone(cnt.get('EmphEnd')))   
    
        # чистка от None 
        for i in range(len(words_snt) - 1, -1, -1):
            if words_snt[i] is None:
                words_snt.pop(i)
                features.pop(i)

        y_snt = np.zeros(len(words_snt), dtype = int)
        y_snt[-1] = 1

        # объединение предложений в один массив
        if bySentence:
            X.append(features)
            y.append(y_snt)
            words.append(words_snt)
        else:
            X.extend(features)            
            y.extend(y_snt)           
            words.extend(words_snt)

    if bySentence == False:
        y = np.asarray(y)
        X = np.asarray(X)
        X = add_n_gramm(n_gramm, X)
    
    return X, y, words

In [8]:
X_train_1, y_train_1, words_1 = get_data(path + book2, n_gramm=2)

In [9]:
X_train_2, y_train_2, words_2 = get_data(path + book3, n_gramm=2)

In [10]:
# Объединение корпусов (2 и 3)

X_train = np.vstack((X_train_1, X_train_2))
y_train = np.vstack((y_train_1.reshape(-1, 1), y_train_2.reshape(-1, 1))).reshape(-1, )

In [11]:
# Тестовый корпус (1)

X_test, y_test, words = get_data(path + book1, n_gramm=2)

# Сложные предложения

In [12]:
def get_sentence(num, X_snt, y_snt, words_snt):
    X = [ft for ft in X_snt[num]]
    y = [y for y in y_snt[num]]
    words = [wrd for wrd in words_snt[num]]
    X = np.asarray(X)
    y = np.asarray(y) 
    return X, y, words

In [13]:
def add_n_gramm_diff(num, n_gramm, X_snt, y_snt, words_snt):
    X = []
    y = []
    words = []
    len = []
    for i in range(-1, 2):
        X_s, y_s, w_s = get_sentence(num + i, X_snt, y_snt, words_snt)
        X.extend(X_s)
        y.extend(y_s)
        words.extend(w_s)
        len.append(X_s.shape[0])

    X = np.asarray(X)    
    X = add_n_gramm(n_gramm, X)
    X = X[len[0]:(len[0] + len[1])]
    y = y[len[0]:(len[0] + len[1])]
    words =  words[len[0]:(len[0] + len[1])]    

    return X, y, words

In [14]:
# Корпуса, где признаки слов разделены по преложениям

X_snt_1, y_snt_1, words_snt_1 = get_data(path + book1, bySentence=True)
X_snt_3, y_snt_3, words_snt_3 = get_data(path + book3, bySentence=True)

In [73]:
# Сложное предложение

X_diff, y_diff, words_diff = add_n_gramm_diff(47, 2, X_snt_1, y_snt_1, words_snt_1)

In [15]:
# Сложные предложения
# book1: 14, 47; book3: 4209

y_diff = []
words_diff = []

X_diff_1, y_diff_1, words_diff_1 = add_n_gramm_diff(14, 2, X_snt_1, y_snt_1, words_snt_1)
y_diff.extend(y_diff_1)
words_diff.extend(words_diff_1)

X_diff_2, y_diff_2, words_diff_2 = add_n_gramm_diff(47, 2, X_snt_1, y_snt_1, words_snt_1)
X_diff = np.vstack((X_diff_1, X_diff_2))
y_diff.extend(y_diff_2)
words_diff.extend(words_diff_2)

X_diff_3, y_diff_3, words_diff_3 = add_n_gramm_diff(4208, 2, X_snt_3, y_snt_3, words_snt_3)
X_diff = np.vstack((X_diff, X_diff_3))
y_diff.extend(y_diff_3)
words_diff.extend(words_diff_3)

X_diff_4, y_diff_4, words_diff_4 = add_n_gramm_diff(4209, 2, X_snt_3, y_snt_3, words_snt_3)
X_diff = np.vstack((X_diff, X_diff_4))
y_diff.extend(y_diff_4)
words_diff.extend(words_diff_4)

In [16]:
_, ys, ws = get_sentence(47, X_snt_1, y_snt_1, words_snt_1)
print(ws)
print(ys)

['Еще', 'не', 'хватает', 'здесь', 'пивного', 'ларька,', 'чтоб', 'эти', 'чубатые', 'вытянувшиеся', 'ребята -', 'порода', 'наша', 'не', 'мельчает! -', 'сдували', 'бы', 'белую', 'пену', 'на', 'могилы.']
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [17]:
_, ys, ws = get_sentence(4209, X_snt_3, y_snt_3, words_snt_3)
print(ws)
print(ys)

['14-го', 'дек.', '1918', 'г.', '4', 'ч.', 'дня".']
[0 0 0 0 0 0 1]


#Классификаторы

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier 

from sklearn.model_selection import cross_val_score
from sklearn import metrics
import matplotlib.pyplot as plt
import pandas as pd

In [19]:
def fit_clf(clf, X_train, y_train):
    clf.fit(X_train, y_train)
    return clf

def predict_clf(clf, X_test, y_test, result = False):
    y_pred = clf.predict(X_test)

    print('F1-score (binary):\t%0.3f' % metrics.f1_score(y_test, y_pred, average='binary'))
    print('Recall (binary): \t%0.3f' % metrics.recall_score(y_test, y_pred, average='binary'))
    print('Precision (binary):\t%0.3f' % metrics.precision_score(y_test, y_pred, average='binary'))
    print('Accuracy:\t\t%0.3f' % metrics.accuracy_score(y_test, y_pred))
    
    print('\nConfusion matrix:\n', pd.DataFrame(metrics.confusion_matrix(y_test, y_pred), index=['0', '1'], columns=['0', '1']))
    #metrics.plot_confusion_matrix(clf, X_test, y_test, values_format='d')
    #plt.show()

    if result:
        return y_pred

def cross_val_clf(clf, X, y):
    scores = cross_val_score(estimator=clf, X=X_train, y=y_train, cv=5, scoring='f1')
    df_scores = pd.DataFrame({'Srocc Val Scores (F1)': scores}, index=['1', '2', '3', '4', '5'])
    print(df_scores)
    print('Average:\t%0.3f\n' % np.mean(scores))

In [20]:
import warnings
warnings.filterwarnings('ignore')

## Decision Tree

In [21]:
tree = DecisionTreeClassifier()
    
cross_val_clf(tree, X_train, y_train)

tree = fit_clf(tree, X_train, y_train)
predict_clf(tree, X_test, y_test)

   Srocc Val Scores (F1)
1               0.993944
2               0.992922
3               0.995203
4               0.980978
5               0.982854
Average:	0.989

F1-score (binary):	0.984
Recall (binary): 	0.989
Precision (binary):	0.979
Accuracy:		0.998

Confusion matrix:
       0   1
0  1160   2
1     1  93


In [23]:
predict_clf(tree, X_diff, y_diff, result=True)

F1-score (binary):	1.000
Recall (binary): 	1.000
Precision (binary):	1.000
Accuracy:		1.000

Confusion matrix:
      0  1
0  105  0
1    0  4


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1])

## Random Forest

In [24]:
rf = RandomForestClassifier(n_estimators=10)

cross_val_clf(rf, X_train, y_train)

rf = fit_clf(rf, X_train, y_train)
predict_clf(rf, X_test, y_test)

   Srocc Val Scores (F1)
1               0.988701
2               0.986694
3               0.989498
4               0.974242
5               0.972850
Average:	0.982

F1-score (binary):	0.967
Recall (binary): 	0.947
Precision (binary):	0.989
Accuracy:		0.995

Confusion matrix:
       0   1
0  1161   1
1     5  89


In [25]:
predict_clf(rf, X_diff, y_diff, result=True)

F1-score (binary):	1.000
Recall (binary): 	1.000
Precision (binary):	1.000
Accuracy:		1.000

Confusion matrix:
      0  1
0  105  0
1    0  4


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1])

## Naive Baies

In [26]:
nb = GaussianNB(var_smoothing=1e-5)

cross_val_clf(nb, X_train, y_train)

nb = fit_clf(nb, X_train, y_train)
predict_clf(nb, X_test, y_test)

   Srocc Val Scores (F1)
1               0.800386
2               0.784450
3               0.815916
4               0.757515
5               0.751063
Average:	0.782

F1-score (binary):	0.780
Recall (binary): 	0.851
Precision (binary):	0.721
Accuracy:		0.964

Confusion matrix:
       0   1
0  1131  31
1    14  80


In [27]:
predict_clf(nb, X_diff, y_diff, result=True)

F1-score (binary):	0.462
Recall (binary): 	0.750
Precision (binary):	0.333
Accuracy:		0.936

Confusion matrix:
     0  1
0  99  6
1   1  3


array([0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

## Logisric Regression

In [28]:
lr = LogisticRegression()

cross_val_clf(lr, X_train, y_train)

lr = fit_clf(lr, X_train, y_train)
predict_clf(lr, X_test, y_test)

   Srocc Val Scores (F1)
1               0.881639
2               0.872214
3               0.877551
4               0.812631
5               0.824354
Average:	0.854

F1-score (binary):	0.896
Recall (binary): 	0.872
Precision (binary):	0.921
Accuracy:		0.985

Confusion matrix:
       0   1
0  1155   7
1    12  82


In [29]:
predict_clf(lr, X_diff, y_diff, result=True)

F1-score (binary):	0.750
Recall (binary): 	0.750
Precision (binary):	0.750
Accuracy:		0.982

Confusion matrix:
      0  1
0  104  1
1    1  3


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

## KNN

In [30]:
knn = KNeighborsClassifier(n_neighbors=5)

cross_val_clf(knn, X_train, y_train)

knn = fit_clf(knn, X_train, y_train)
predict_clf(knn, X_test, y_test)

   Srocc Val Scores (F1)
1               0.733127
2               0.724360
3               0.734315
4               0.615432
5               0.639537
Average:	0.689

F1-score (binary):	0.658
Recall (binary): 	0.564
Precision (binary):	0.791
Accuracy:		0.956

Confusion matrix:
       0   1
0  1148  14
1    41  53


In [31]:
predict_clf(knn, X_diff, y_diff, result=True)

F1-score (binary):	0.571
Recall (binary): 	0.500
Precision (binary):	0.667
Accuracy:		0.972

Confusion matrix:
      0  1
0  104  1
1    2  2


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])